In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [12]:

try:
    df = pd.read_csv('/content/winequality-red.csv')
except FileNotFoundError:
    print("Error: winequality-red.csv not found!")

maxdepth= 8

train_data , test_data = train_test_split(df,test_size=0.1,random_state=12)

for dataset in [train_data, test_data]:
    if dataset['quality'].dtype != 'object':
        original_quality = dataset['quality'].copy()
        dataset.loc[original_quality <= 5, 'quality'] = 'bad'
        dataset.loc[original_quality >= 6, 'quality'] = 'good'







/tmp/ipython-input-12-404004229.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'bad' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dataset.loc[original_quality <= 5, 'quality'] = 'bad'
/tmp/ipython-input-12-404004229.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'bad' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dataset.loc[original_quality <= 5, 'quality'] = 'bad'


#EDA

1.dataset overview

2.Target variable distribution



In [15]:
dataset.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [16]:
dataset.shape

(160, 12)

No null columns therefore there is no data cleaning that must be performed

In [22]:
dataset.isnull().sum().sum()

np.int64(0)

In [24]:
dataset['quality'].value_counts().sort_index()

,count
quality,
bad,74
good,86


#Node Class
constructor with many local variables
#Functions
split children
gini impurity
find best split
predict
stop

In [13]:

class Node():
    def __init__(self,data,features,depth=0):
        self.depth=depth
        self.data=data
        self.good=len(self.data[self.data['quality']=='good'])
        self.bad=len(self.data[self.data['quality']=='bad'])
        self.isleaf=False
        self.featurelist= features
        self.rightdata= 0
        self.leftdata=0
        self.split_feature = None
        self.split_threshold = None
        if self.depth>= maxdepth:
             self.stop()
        else:
            self.split_children()




    # Functions


    def split_children(self):
        feature=self.featurelist[0]
        data1,data2=[None],[None]
        self.split_feature = feature
        self.find_best_split(data1,data2,feature)
        newftlist= self.featurelist.drop(feature)
        if data1[0] is not None and data2[0] is not None:
            self.right_child = Node(data1[0], newftlist, self.depth+1)
            self.left_child = Node(data2[0], newftlist, self.depth+1)
        else:
             self.stop()



    def gini_impurity(self,good,bad):
        if good==0 or bad==0:
            return 0

        return 1 - ((good/(bad+good))**2) - ((bad/(bad+good))**2)



    def find_best_split(self,data1,data2,feature):
        bestgini=10000
        threshold = None
        bestleft= None
        bestright= None

        feature_unique_len = (self.data[feature].unique())
        for i in feature_unique_len:
            left = self.data[self.data[feature]>i]
            right = self.data[self.data[feature]<=i]
            left_gini = self.gini_impurity(len(left[left['quality']=='good']),len(left[left['quality']=='bad']))
            right_gini = self.gini_impurity(len(right[right['quality']=='good']),len(right[right['quality']=='bad']))
            total = len(left) + len(right)
            gini = (len(left)/total)*left_gini + (len(right)/total)*(right_gini)

            if gini<bestgini:
                bestgini=gini
                bestleft=left
                bestright=right
                threshold = i


        self.split_threshold= threshold

        data1[0]=bestleft
        data2[0]=bestright
        return bestgini


    def predict(self,wine_sample):
        if self.isleaf:
            if self.good > self.bad:
                return "good"
            else:
                return "bad"

        wine_feature_value = wine_sample[self.split_feature]
        if wine_feature_value <= self.split_threshold:
            return self.left_child.predict(wine_sample)
        else:
            return self.right_child.predict(wine_sample)



    def stop(self):
        self.isleaf=True

        total = len(test_data)

#Main function
Prints some sample test data predictions along witht he actual value and shows the baseline accuracy along with increased accuracy due to the decision tree
with depth 8

In [14]:
def main():
    tree = build_tree()
    print("--- Decision Tree Validation ---")

    correct = 0
    total = len(test_data)


    for i in range(total):
        sample = test_data.iloc[i]
        actual = sample['quality']
        predicted = tree.predict(sample)

        if predicted == actual:
            correct += 1


        if i < 10:
            match = "✓" if predicted == actual else "✗"
            print(f"Sample {i}: Predicted={predicted}, Actual={actual} {match}")


    accuracy = (correct / total) * 100
    print(f"\n--- Results ---")
    print(f"Correct predictions: {correct}/{total}")
    print(f"Accuracy: {accuracy:.1f}%")


    good_count = len(test_data[test_data['quality'] == 'good'])
    baseline = max(good_count, total - good_count) / total * 100
    print(f"Baseline (guessing majority): {baseline:.1f}%")

main()

--- Decision Tree Validation ---
Sample 0: Predicted=bad, Actual=good ✗
Sample 1: Predicted=good, Actual=good ✓
Sample 2: Predicted=good, Actual=good ✓
Sample 3: Predicted=bad, Actual=bad ✓
Sample 4: Predicted=bad, Actual=good ✗
Sample 5: Predicted=bad, Actual=bad ✓
Sample 6: Predicted=bad, Actual=bad ✓
Sample 7: Predicted=bad, Actual=bad ✓
Sample 8: Predicted=bad, Actual=bad ✓
Sample 9: Predicted=good, Actual=good ✓

--- Results ---
Correct predictions: 120/160
Accuracy: 75.0%
Baseline (guessing majority): 53.8%


#Checks the accuracy change
with less depth (depth=4)

In [25]:
maxdepth= 4
main()

--- Decision Tree Validation ---
Sample 0: Predicted=good, Actual=good ✓
Sample 1: Predicted=bad, Actual=good ✗
Sample 2: Predicted=bad, Actual=good ✗
Sample 3: Predicted=bad, Actual=bad ✓
Sample 4: Predicted=good, Actual=good ✓
Sample 5: Predicted=bad, Actual=bad ✓
Sample 6: Predicted=bad, Actual=bad ✓
Sample 7: Predicted=bad, Actual=bad ✓
Sample 8: Predicted=bad, Actual=bad ✓
Sample 9: Predicted=good, Actual=good ✓

--- Results ---
Correct predictions: 99/160
Accuracy: 61.9%
Baseline (guessing majority): 53.8%


# accuracy change with higher depth

 (depth=9) some information loss has occured

In [26]:
maxdepth=9
main()

--- Decision Tree Validation ---
Sample 0: Predicted=bad, Actual=good ✗
Sample 1: Predicted=good, Actual=good ✓
Sample 2: Predicted=good, Actual=good ✓
Sample 3: Predicted=bad, Actual=bad ✓
Sample 4: Predicted=good, Actual=good ✓
Sample 5: Predicted=bad, Actual=bad ✓
Sample 6: Predicted=bad, Actual=bad ✓
Sample 7: Predicted=bad, Actual=bad ✓
Sample 8: Predicted=bad, Actual=bad ✓
Sample 9: Predicted=good, Actual=good ✓

--- Results ---
Correct predictions: 119/160
Accuracy: 74.4%
Baseline (guessing majority): 53.8%


#Results
maxdepth = 8 gives best accuracy 75%

any more max depth=9 accuracy decreases to 74.4%

and and less maxdepth = 4 accuracy decreases to 61.9%